In [1]:
import pandas as pd
import numpy as np
import os
import sys

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
cwd = os.getcwd()

education_folder_path = os.path.dirname(cwd)

sys.path.append(education_folder_path)

In [3]:
from global_functions import remove_spaces_in_columns, up_load_df

In [4]:
#בתי ספר וגנים מעיריית ירושלים
JLM=up_load_df(r'{}\background_files\jerusalem_muni'.format(education_folder_path),'מוסדות בירושלים 2020')
JLM = remove_spaces_in_columns(JLM)

In [5]:
JLM.rename(columns={'סמל_חינוך': 'סמל_מוסד'}, inplace=True)

JLM = JLM.drop(columns=['שם_מוסד', 'coordinate_x', 'coordinate_y', 'אגף', 'פיקוח', 'סוג_חינוך', 'מעמד_משפטי', 'מספר_כיתות', "מס'_תלמידים_ז-ט", "מס'_תלמידים_י-יד", 'קוד_אזור_סטיסטי', 'תאור_אזור_סטטיסטי', 'Unnamed:_18', 'Unnamed:_19'])

JLM.head()

,סמל_עירייה,סמל_מוסד,שלב_חינוך,מכיתה,עד_כיתה,"סה""כ_תלמידים"
0,21321.00,110106.00,יסודי,א,ו,230.00
1,21323.00,110312.00,יסודי,א,ח,347.00
2,21326.00,110155.00,יסודי,א,ו,534.00
3,21327.00,110171.00,יסודי,א,ו,331.00
4,21328.00,111112.00,יסודי,א,ו,367.00


In [6]:
# פונקציה להמרת כיתות לאותיות למספרים
def get_grade_number(grade):
    grade_map = {
        '3': 3, '4': 4, '5': 5, 'א': 6, 'ב': 7, 'ג': 8, 'ד': 9, 'ה': 10, 'ו': 11, 'ז': 12, 'ח': 13, 'ט': 14, 'י': 15, 'יא': 16, 'יב': 17, 'יג': 18, 'יד': 19,
    }
    return grade_map.get(grade, np.nan)

In [7]:
# חישוב מספר הכיתות
JLM['מספר_כיתות'] = JLM.apply(lambda row: get_grade_number(row['עד_כיתה']) - get_grade_number(row['מכיתה']) + 1, axis=1)
JLM['מספר_כיתות']

# חלוקת התלמידים לפי כיתות
JLM['תלמידים_לכיתה'] = JLM['סה"כ_תלמידים'] / JLM['מספר_כיתות']

In [8]:
# יצירת עמודות עבור כל כיתה
grades = ['א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'יא', 'יב', 'יג', 'יד']
for grade in grades:
    JLM[grade] = JLM.apply(lambda row: row['תלמידים_לכיתה'] if get_grade_number(row['מכיתה']) <= get_grade_number(grade) <= get_grade_number(row['עד_כיתה']) else 0, axis=1)

# JLM.head(600)

In [9]:
# מיפוי הכיתות לפי שכבות גיל
ages = {
    'kgarden': [3, 4, 5],
    'ele': ['א', 'ב', 'ג', 'ד', 'ה', 'ו'],
    'mid': ['ז', 'ח', 'ט'],
    'high': ['י', 'יא', 'יב', 'יג', 'יד']
}

# פונקציה להמרת אותיות למספרים עבור השדה 'מכיתה' ו'עד כיתה'
def class_to_num(c):
    classes = {'א': 6, 'ב': 7, 'ג': 8, 'ד': 9, 'ה': 10, 'ו': 11, 'ז': 12, 'ח': 13, 'ט': 14, 'י': 15, 'יא': 16, 'יב': 17, 'יג': 18, 'יד': 19}
    return classes.get(c, c)

JLM['מכיתה_num'] = JLM['מכיתה'].apply(class_to_num)
JLM['עד_כיתה_num'] = JLM['עד_כיתה'].apply(class_to_num)

# פונקציה לחישוב מספר הכיתות לכל שכבת גיל
def calc_classes_for_age(row, age_group):
    # קבלת טווח הכיתות עבור כל שכבת גיל
    if age_group == 'kgarden':
        # גן ילדים
        return sum([1 for x in ages['kgarden'] if row['מכיתה_num'] <= x <= row['עד_כיתה_num']])
    elif age_group == 'ele':
        # יסודי
        return sum([1 for x in ages['ele'] if class_to_num(x) >= row['מכיתה_num'] and class_to_num(x) <= row['עד_כיתה_num']])
    elif age_group == 'mid':
        # חטיבת ביניים
        return sum([1 for x in ages['mid'] if class_to_num(x) >= row['מכיתה_num'] and class_to_num(x) <= row['עד_כיתה_num']])
    elif age_group == 'high':
        # על יסודי
        return sum([1 for x in ages['high'] if class_to_num(x) >= row['מכיתה_num'] and class_to_num(x) <= row['עד_כיתה_num']])
    return 0

# הוספת עמודות לכל שכבת גיל
JLM['kgarden_class'] = JLM.apply(calc_classes_for_age, axis=1, age_group='kgarden')
JLM['ele_class'] = JLM.apply(calc_classes_for_age, axis=1, age_group='ele')
JLM['mid_class'] = JLM.apply(calc_classes_for_age, axis=1, age_group='mid')
JLM['high_class'] = JLM.apply(calc_classes_for_age, axis=1, age_group='high')

JLM = JLM.drop(columns=['מכיתה_num', 'עד_כיתה_num'])

# JLM.head(600)

In [10]:
# פונקציה לחישוב מספר התלמידים לפי שכבת גיל
def calculate_students_by_class(row):
    if row['שלב_חינוך'] == 'גני ילדים':
        row['kgarden_stu'] = row['סה"כ_תלמידים']
    else:
        row['kgarden_stu'] = 0
    
    # חישוב תלמידים לשכבת יסודי
    row['ele_stu'] = row[ages['ele']].sum()
    
    # חישוב תלמידים לשכבת חטיבת ביניים
    row['mid_stu'] = row[ages['mid']].sum()
    
    # חישוב תלמידים לשכבת תיכון
    row['high_stu'] = row[ages['high']].sum()
    
    return row

# עדכון כל שורה בטבלה עם הנתונים החדשים
JLM = JLM.apply(calculate_students_by_class, axis=1)

# JLM.head(600)

In [14]:
# # הצגת התוצאה
JLM = JLM.drop(columns=['סמל_עירייה', 'תלמידים_לכיתה', 'מספר_כיתות'])

# JLM.head(600)

JLM.to_excel('data.xlsx', index=False)